In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import sys
import os
from subprocess import call 
import pathlib

In [4]:
import numpy as np
import pandas as pd
import xarray as xr

In [5]:
from dask.distributed import Client
from dask.diagnostics import ProgressBar

In [6]:
sys.path.append("../code/")

In [7]:
import src

### parameters for papermill

In [8]:
domain = 'Ninos'
ndays = 30
roll = 15
# climatology = [1991, 2020]
climatology = [1961, 1990]
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]

### create cluster

In [9]:
client = Client(
    n_workers=6, threads_per_worker=4, memory_limit="8GB", local_directory="./dask"
)

In [10]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 44.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40787,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 44.70 GiB
Comm: tcp://127.0.0.1:37951,Total threads: 4
Dashboard: http://127.0.0.1:44599/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:33277,


In [11]:
sys.path.append('../code/')

In [12]:
import src

In [13]:
opath = pathlib.Path(f'/media/nicolasf/END19101/data/OISST/daily/{domain}')

In [14]:
lfiles = list(opath.glob("sst.day.mean.????.v2.nc"))

In [15]:
lfiles.sort()

In [16]:
lfiles[0]

PosixPath('/media/nicolasf/END19101/data/OISST/daily/Ninos/sst.day.mean.1981.v2.nc')

In [17]:
lfiles[-1]

PosixPath('/media/nicolasf/END19101/data/OISST/daily/Ninos/sst.day.mean.2022.v2.nc')

In [18]:
dset = xr.open_mfdataset(lfiles, parallel=True, combine_attrs="drop_conflicts", compat="override", coords=['time'])

In [19]:
dset

<xarray.Dataset>
Dimensions:  (time: 14954, lon: 360, lat: 60)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2022-08-10
  * lon      (lon) float32 190.1 190.4 190.6 190.9 ... 279.1 279.4 279.6 279.9
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 4.375 4.625 4.875
Data variables:
    sst      (time, lat, lon) float32 dask.array<chunksize=(122, 60, 360), meta=np.ndarray>

### if regions is 'Ninos' or 'IOD', then first calculates the regional averages

In [20]:
if domain == 'Ninos':
    
    dset['sst'] = src.calculates_ninos(dset['sst'], nino='all')
    
elif domain == 'IOD': 
    
    dset['sst']= src.calculates_IOD_nodes(dset['sst'], IOD_node='all')

In [21]:
dset

<xarray.Dataset>
Dimensions:  (time: 14954, lon: 360, lat: 60, nino: 5)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2022-08-10
  * lon      (lon) float32 190.1 190.4 190.6 190.9 ... 279.1 279.4 279.6 279.9
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 4.375 4.625 4.875
  * nino     (nino) object '1+2' '3' '4' '3.4' 'oni'
Data variables:
    sst      (nino, time) float32 dask.array<chunksize=(1, 122), meta=np.ndarray>

### remove the 29th of Feb

In [22]:
# standard_calendar = pd.date_range(start=first_day, end=last_day, freq="D")

In [23]:
dset = dset.convert_calendar('noleap')

### calculates the rolling averages if needed 

In [24]:
if ndays > 1: 
    
    dset['sst'] = dset['sst'].rolling({'time':ndays}, center=False, min_periods=ndays).mean('time')
    
    dset = dset.isel(time=slice(ndays+1,None))

In [25]:
dset

<xarray.Dataset>
Dimensions:  (nino: 5, time: 14913, lon: 360, lat: 60)
Coordinates:
  * time     (time) object 1981-10-02 00:00:00 ... 2022-08-10 00:00:00
  * lon      (lon) float32 190.1 190.4 190.6 190.9 ... 279.1 279.4 279.6 279.9
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 4.375 4.625 4.875
  * nino     (nino) object '1+2' '3' '4' '3.4' 'oni'
Data variables:
    sst      (nino, time) float64 dask.array<chunksize=(1, 120), meta=np.ndarray>

### now expands the dataset along a dummy dimension 

In [26]:
dset_roll = dset[['sst']].copy()

In [27]:
dset_roll = dset_roll.rolling({'time':roll}, center=True, min_periods=roll).construct(window_dim='roll')

### selects the climatological period 

In [28]:
clim = dset_roll.sel(time=slice(str(climatology[0]), str(climatology[1])))

In [29]:
clim

<xarray.Dataset>
Dimensions:  (time: 3376, nino: 5, roll: 15)
Coordinates:
  * time     (time) object 1981-10-02 00:00:00 ... 1990-12-31 00:00:00
  * nino     (nino) object '1+2' '3' '4' '3.4' 'oni'
Dimensions without coordinates: roll
Data variables:
    sst      (nino, time, roll) float64 dask.array<chunksize=(1, 127, 15), meta=np.ndarray>

### transpose, then rechunks 

In [30]:
if domain == 'Ninos': 
    
    clim = clim.transpose(*['time','roll','nino'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'nino':len(clim['nino'])})
    
elif domain == 'IOD': 

    clim = clim.transpose(*['time','roll','IOD'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'IOD':len(clim['IOD'])})
    
else: 
    
    clim = clim.transpose(*['time','roll','lat','lon'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'lat':2, 'lon':20})

In [31]:
clim

<xarray.Dataset>
Dimensions:  (time: 3376, nino: 5, roll: 15)
Coordinates:
  * time     (time) object 1981-10-02 00:00:00 ... 1990-12-31 00:00:00
  * nino     (nino) object '1+2' '3' '4' '3.4' 'oni'
Dimensions without coordinates: roll
Data variables:
    sst      (time, roll, nino) float64 dask.array<chunksize=(3376, 15, 5), meta=np.ndarray>

In [32]:
clim_q = clim.groupby(clim.time.dt.dayofyear).quantile(quantiles, dim=['time','roll'])

In [33]:
clim_ave = clim.groupby(clim.time.dt.dayofyear).mean(dim=['time','roll'])

In [34]:
clim_std = clim.groupby(clim.time.dt.dayofyear).std(dim=['time','roll'])

In [35]:
with ProgressBar(): 
    clim_q = clim_q.compute()
    clim_ave = clim_ave.compute()
    clim_std = clim_std.compute()

In [36]:
clim_q = clim_q.rename({'sst':'quantiles'})

In [37]:
clim_ave

<xarray.Dataset>
Dimensions:    (nino: 5, dayofyear: 365)
Coordinates:
  * nino       (nino) object '1+2' '3' '4' '3.4' 'oni'
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Data variables:
    sst        (nino, dayofyear) float64 23.06 23.11 23.16 ... 26.39 26.39 26.39

In [38]:
clim_q['average'] = clim_ave['sst']

In [39]:
clim_q

<xarray.Dataset>
Dimensions:    (nino: 5, dayofyear: 365, quantile: 5)
Coordinates:
  * nino       (nino) object '1+2' '3' '4' '3.4' 'oni'
  * quantile   (quantile) float64 0.1 0.25 0.5 0.75 0.9
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Data variables:
    quantiles  (dayofyear, quantile, nino) float64 22.03 23.51 ... 27.59 27.59
    average    (nino, dayofyear) float64 23.06 23.11 23.16 ... 26.39 26.39 26.39

In [40]:
clim_q['std'] = clim_std['sst']

In [41]:
opath = pathlib.Path(f'../outputs/{domain}')

In [42]:
opath.mkdir(exist_ok=True)

### saves to ZARR

In [43]:
clim_q.to_zarr(opath.joinpath(f'{domain}_OISST_{ndays}days_climatology_{roll}_window_1961_1990.zarr'))